In [1]:
import random
import pandas as pd
from datetime import datetime, timedelta

In [2]:
def generate_case(tasks, case_id, start_time,min_datapoints):
    case = []
    time = start_time#100
    case_id=str(case_id).zfill(len(str(min_datapoints)))
    case.append(['null','S0',time,'Req',case_id])
    
    for task,subtasks in tasks.items():
        server = f"S{list(tasks.keys()).index(task)+1}" #SX
        case.append(['S0',server,time,'Req',case_id])
        if subtasks[0]=='one':
            #print('one')
            service_task = random.choice(subtasks[1])
            #print(service_task)
            new_server = f"{server}_{subtasks[1].index(service_task)+1}"
            case.append([server, new_server,time,'Req',case_id])
            #Response
            time += timedelta(microseconds=random.randint(subtasks[-1][0], subtasks[-1][1])*100000)
            case.append([new_server,server,time,'Res',case_id]) #1 milisecond= 1000 microseconds
            #Request
            

        elif subtasks[0] == 'rand':
            #print('rand')
            rand_tasks = random.sample(subtasks[1], random.randint(1, len(subtasks[1])))
            for new_task in rand_tasks:
                #print(new_task)
                new_server = f"{server}_{subtasks[1].index(new_task)+1}"
                #Request
                case.append([server, new_server,time,'Req',case_id])
                #Response
                #time += timedelta(microseconds=subtasks[-1]*1000)
                time += timedelta(microseconds=random.randint(subtasks[-1][0], subtasks[-1][1])*100000)
                case.append([new_server,server,time,'Res',case_id]) #1 milisecond= 1000 microseconds
            #print(rand_tasks)
            #print('#####')

        elif subtasks[0] == 'all':
            #print('all')
            for new_task in subtasks[1]:
                #print(new_task)
                new_server = f"{server}_{subtasks[1].index(new_task)+1}"
                #Request
                case.append([server, new_server,time,'Req',case_id])
                #Response
                #time += timedelta(microseconds=subtasks[-1]*1000)
                time += timedelta(microseconds=random.randint(subtasks[-1][0], subtasks[-1][1])*100000)
                case.append([new_server,server,time,'Res',case_id]) #1 milisecond= 1000 microseconds
            #print('#####')

        else:
            print('smth wrong')
            #raise Error
        #print(subtask)
        
        case.append([server,'S0',time,'Res',case_id])
    case.append(['S0','null',time,'Res',case_id])
    return case

#print(generate_case(tasks, user_id, start_time))

In [3]:
#Random times
def random_time(start_time,end_time):
    delta = end_time - start_time
    return start_time + timedelta(seconds=random.randint(0, int(delta.total_seconds())))

In [8]:
def generate_dataset(tasks, min_datapoints,start_time,end_time):
    datapoints = 0
    dataset = []
    user_id = 1
    while datapoints <= min_datapoints:
        time = random_time(start_time,end_time)
        case = generate_case(tasks, user_id, time,min_datapoints)
        dataset.extend(case)
        datapoints+= len(case)
        user_id+=1
    
    df = pd.DataFrame(dataset, columns=['from', 'to', 'timestamp', 'type', 'user_id'])

    # Save to Excel (XLSX)
    df.to_csv('./SGD file Tomas.csv', index=False)  # Save to XLSX instead of CSV

    # Display sample of the dataset
    print(df.head())

tasks = {'log_in': ['one',['credentials check','sing up','recover pw and log in'],(0,10)],
            'search_book': ['rand',['history','fantasy','crime','poetry','biography'],(5,15)],
            'shipment' : ['all',['adress','door number','zip code'],(15,50)],
            'payment' : ['one',['visa','master card','revolut','paypal','apple pay'],(10,100)]
}  
start_time = datetime(2024, 6, 3, 9, 0, 0)  #begining of the course April 27, 2024, 08:00:00
end_time = datetime(2024, 6, 3, 10, 45, 0) #day of the exam
generate_dataset(tasks, 1000,start_time,end_time)  

   from    to           timestamp type user_id
0  null    S0 2024-06-03 09:50:08  Req    0001
1    S0    S1 2024-06-03 09:50:08  Req    0001
2    S1  S1_2 2024-06-03 09:50:08  Req    0001
3  S1_2    S1 2024-06-03 09:50:08  Res    0001
4    S1    S0 2024-06-03 09:50:08  Res    0001


In [5]:
# def generate_transfers_dataset(tasks, min_datapoints,start_time,end_time):
#     datapoints = 0
#     dataset = []
#     user_id = 1
#     while datapoints <= min_datapoints:
#         time = random_time(start_time,end_time)
#         case = generate_case(tasks, user_id, time)
#         dataset.extend(case)
#         datapoints+= len(case)
#         user_id+=1
    
#     df = pd.DataFrame(dataset, columns=['from', 'to', 'timestamp', 'type', 'user_id'])

#     # Save to Excel (XLSX)
#     df.to_csv('./SGD_books.csv', index=False)  # Save to XLSX instead of CSV
#     #df.to_csv('./SGD_transfers.csv', index=False)  # Save to XLSX instead of CSV

#     # Display sample of the dataset
#     print(df.head())

# tasks = {
#     'log_in': ['one', ['credentials_check', 'sign_up', 'recover_password_ad_log_in'],3],
#     'initiate_transfer': ['all', ['enter_account_details', 'enter_amount', 'enter_recipient_details'],3],
#     'complete_transfer': ['one', ['Visa', 'Master_card', 'Revolut', 'Paypal'],4],
#     'verify_transfer': ['rand', ['sms_verification', 'email_verification', '2Fauthenticator_verification'],3]
# }

# start_time = datetime(2024, 4, 22, 9, 0, 0)  #begining of the course April 27, 2024, 08:00:00
# end_time = datetime(2024, 6, 18, 15, 0, 0) #day of the exam

In [6]:
#generate_book_dataset(tasks, 100,start_time,end_time)  

In [7]:
#generate_transfers_dataset(tasks, 1000000,start_time,end_time)  